In [8]:
def sliding_window(image, window_size, step_size):
    for y in range(0, image.shape[0], step_size[1]):
        for x in range(0, image.shape[1], step_size[0]):
            yield (x, y, image[y: y + window_size[1], x: x + window_size[0]])

In [9]:
import sys
sys.path.append(r'C:\Users\bee\Desktop')
import Sliding

In [10]:
import numpy as np 
import cv2,joblib
import Sliding as sd
from imutils.object_detection import non_max_suppression
import imutils
from skimage.feature import hog
from skimage import color
from skimage.transform import pyramid_gaussian


image = cv2.imread(r'C:\Users\bee\Desktop\ppl.jpg')
image = cv2.resize(image,(400,256))
size = (64,128)
step_size = (9,9)
downscale = 1.25
#List to store the detections
detections = []
#The current scale of the image 
scale = 0
model = joblib.load(r'C:\Users\bee\Desktop\models.dat')
for im_scaled in pyramid_gaussian(image, downscale = downscale):
    #The list contains detections at the current scale
    if im_scaled.shape[0] < size[1] or im_scaled.shape[1] < size[0]:
        break
    for (x, y, window) in sd.sliding_window(im_scaled, size, step_size):
        if window.shape[0] != size[1] or window.shape[1] != size[0]:
            continue
        window = color.rgb2gray(window)
            
        fd=hog(window, orientations=9,pixels_per_cell=(8,8),visualize=False,cells_per_block=(3,3))
        fd = fd.reshape(1, -1)
        pred = model.predict(fd)
        if pred == 1:
                
            if model.decision_function(fd) > 0.5:
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fd), 
                int(size[0] * (downscale**scale)),
                int(size[1] * (downscale**scale))))
 
    scale += 1
clone = image.copy()
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections])
sc = [score[0] for (x, y, score, w, h) in detections]
print ("sc: ", sc)
sc = np.array(sc)
pick = non_max_suppression(rects, probs = sc, overlapThresh = 0.3)
for(x1, y1, x2, y2) in pick:
    cv2.rectangle(clone, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(clone,'Person',(x1-2,y1-2),1,0.75,(121,12,34),1)
cv2.imshow('Person Detection',clone)
cv2.waitKey(0)
cv2.destroyAllWindows()


C:\Users\bee\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


sc:  [1.3193242569312953, 0.9269982253057756, 0.6501839594583312, 2.0484493499712224, 0.5435326220906433, 0.9308168070109999, 1.1446440033271008, 0.5572674958680648, 0.7022031115837724, 0.7314457685488911, 1.7351451520453578, 3.3513141950964807, 2.29709903656141, 0.9860116486187209, 0.8952391552679773, 0.7723251790582846, 1.0645772856772537, 1.481582036807798, 0.6032052379240205, 1.1429689688122906, 0.6993035526884785, 1.325804155753206, 1.7094704641645233, 1.1945365812108166, 1.3952797256214406, 0.6058373065989073, 1.2571158024739302, 0.5800096466130453, 0.9323138141658331, 1.627227171661402, 0.9382103906654464, 0.5713176265443756, 1.146186118475799, 1.3695887773813515, 1.2537807364387565, 1.371203852955123, 1.9964170033763287, 1.5854938302241335, 0.9395305896779709, 0.8553222628507218, 1.4897704703236316, 1.1040726815326716, 0.6896473204624094, 2.1834921982241307, 1.2896292817352575, 3.6450548321870597, 3.0763771233855888, 2.5889270944563547, 3.4508976530505517, 1.2692645763287964, 3